# Integration and joint analysis of Xenium and Visium data

Authors: Elyas Heidari, Luca Marconato

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext jupyter_black

## Setup
### Import libraries

In [ ]:
import os
import warnings

os.environ["USE_PYGEOS"] = "0"
warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path

import anndata
import numpy as np
import scanpy as sc
import spatialdata as sd
from spatialdata import aggregate
from spatialdata.transformations import (
    Identity,
    Sequence,
    align_elements_using_landmarks,
    get_transformation,
    set_transformation,
)

# from napari_spatialdata import Interactive

### Data paths

#### Setting up the root data folders

In [ ]:
print("current working directory:", Path().cwd())
SPATIALDATA_SANDBOX_PATH = Path("spatialdata-sandbox")
assert (
    SPATIALDATA_SANDBOX_PATH.is_dir()
), f"{SPATIALDATA_SANDBOX_PATH} not found, please use symlinks to make it available or change the path"
GENERATED_DATA_PATH = SPATIALDATA_SANDBOX_PATH / "generated_data/xenium_visium_integration"
GENERATED_DATA_PATH.mkdir(parents=True, exist_ok=True)

#### Data to be downloaded

scRNA-Seq reference atlas from [Wu et al., Nat. Genet 2021](https://www.nature.com/articles/s41588-021-00911-1). It can be [downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/generated_data/xenium_visium_integration/BC_atlas_xe.h5ad).

In [ ]:
BC_SC_ATLAS_PATH = GENERATED_DATA_PATH / "BC_atlas_xe.h5ad"

Clonal information derived from the Visium data, [can be downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/generated_data/xenium_visium_integration/visium_copyKat.h5ad).

In [ ]:
VISIUM_CLONAL_PATH = GENERATED_DATA_PATH / "visium_copyKat.h5ad"

Zarr file with annotated landmark locations and a ROI polygon, drawn with napari-spatialdata and saved into a SpatialData object, [can be downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/generated_data/xenium_visium_integration/sandbox.zarr.zip).

Note: some software for extracting `.zip` files will create an outer folder called `sandbox.zarr`, resulting in a directory structure like `sandbox.zarr/sandbox.zarr/`. Please ensure that only the inner `sandbox.zarr` is present (the one containing the `.zgroup` file).

In [ ]:
LANDMARKS_SDATA_PATH = GENERATED_DATA_PATH / "sandbox.zarr"

#### Data converted to Zarr with scripts from `spatialdata-sandbox` (can also be downloaded)

To get this data:
1. run spatialdata-sandbox/xenium_rep1_io/download.py;
2. run spatialdata-sandbox/xenium_rep1_io/to_zarr.py to create the zarr files.
Alternatively, it [can be downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/xenium_rep1_io.zip).

In [ ]:
XE_REP1_PATH = SPATIALDATA_SANDBOX_PATH / "xenium_rep1_io/data.zarr"

To get this data:
1. run spatialdata-sandbox/xenium_rep2_io/download.py;
2. run spatialdata-sandbox/xenium_rep2_io/to_zarr.py to create the zarr files.
Alternatively, it [can be downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/xenium_rep2_io.zip).

In [ ]:
XE_REP2_PATH = SPATIALDATA_SANDBOX_PATH / "xenium_rep2_io/data.zarr"

To get this data:
1. run spatialdata-sandbox/visium_associated_xenium_io/download.py
2. run spatialdata-sandbox/visium_associated_xenium_io/to_zarr.py to create the zarr files
Alternatively, it [can be downloaded from here](https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_associated_xenium_io.zip).

In [ ]:
VISIUM_PATH = SPATIALDATA_SANDBOX_PATH / "visium_associated_xenium_io/data.zarr"

#### Checking that all the data paths are available.

In [ ]:
paths = [
    BC_SC_ATLAS_PATH,
    VISIUM_CLONAL_PATH,
    XE_REP1_PATH,
    XE_REP2_PATH,
    VISIUM_PATH,
    LANDMARKS_SDATA_PATH,
]
for path in paths:
    assert path.exists(), f"{path} not found"

#### Data that will be generated by this notebook

The notebook will update in-place some of the root files above, and will also create the following Zarr files

In [ ]:
# output paths for xenium_rep1, xenium_rep2 and visium data, after subsetting to the common area and being transformed
XE_REP1_ROI_PATH = GENERATED_DATA_PATH / "xe_rep1_roi.zarr"
XE_REP2_ROI_PATH = GENERATED_DATA_PATH / "xe_rep2_roi.zarr"
VISIUM_ROI_PATH = GENERATED_DATA_PATH / "visium_roi.zarr"

### Loading the data

In [ ]:
bc_sc_atlas_adata = sc.read(BC_SC_ATLAS_PATH)
bc_sc_atlas_adata.obs["dataset"] = "atlas"

xe_rep1_sdata = sd.read_zarr(XE_REP1_PATH)
xe_rep1_adata = xe_rep1_sdata.table
xe_rep1_adata.obs["dataset"] = "xe_rep1"

xe_rep2_sdata = sd.read_zarr(XE_REP2_PATH)
xe_rep2_adata = xe_rep2_sdata.table
xe_rep2_adata.obs["dataset"] = "xe_rep2"

visium_sdata = sd.read_zarr(VISIUM_PATH)
visium_adata = visium_sdata.table
visium_adata.obs["dataset"] = "visium"

landmarks_sdata = sd.read_zarr(LANDMARKS_SDATA_PATH)
clonal_adata = sc.read(VISIUM_CLONAL_PATH)

In [ ]:
landmarks_sdata

### Integrating annotations

#### Transferring cell-types into Xenium

Let's transfer the cell-types information from an single-cell atlas dataset to the Xenium cells. We will cache the result.

In [ ]:
def annotate_bc_xe(bc_sc_atlas_adata, adata_query):
    genes = list(set(bc_sc_atlas_adata.var_names) & set(adata_query.var_names))
    bc_sc_atlas_adata = bc_sc_atlas_adata[:, genes]
    adata_query = adata_query[:, genes]
    sc.pp.normalize_total(adata_query, target_sum=1e4)
    sc.pp.log1p(adata_query)

    sc.pp.pca(bc_sc_atlas_adata)
    sc.pp.neighbors(bc_sc_atlas_adata)
    sc.tl.umap(bc_sc_atlas_adata)
    sc.tl.ingest(adata_query, bc_sc_atlas_adata, obs="celltype_major")

    ad = {}
    for t in adata_query.obs["celltype_major"].unique():
        query_sub = adata_query[adata_query.obs["celltype_major"] == t]
        ref_sub = bc_sc_atlas_adata[bc_sc_atlas_adata.obs["celltype_major"] == t]
        sc.pp.pca(ref_sub)
        sc.pp.neighbors(ref_sub)
        sc.tl.umap(ref_sub)
        sc.tl.ingest(query_sub, ref_sub, obs="celltype_minor", inplace=True)
        ad[t] = query_sub

    adata_query = anndata.concat(ad)
    return adata_query

In [ ]:
CELL_TYPES_ALREADY_TRANSFERRED = True
if not CELL_TYPES_ALREADY_TRANSFERRED:
    # this code can take 10-60 min to execute
    xe_rep1_annotated = annotate_bc_xe(bc_sc_atlas_adata, xe_rep1_adata)
    xe_rep2_annotated = annotate_bc_xe(bc_sc_atlas_adata, xe_rep2_adata)
    xe_rep1_annotated.write(GENERATED_DATA_PATH / "xe_rep1_annotated.h5ad")
    xe_rep2_annotated.write(GENERATED_DATA_PATH / "xe_rep2_annotated.h5ad")
else:
    xe_rep1_annotated = sc.read(GENERATED_DATA_PATH / "xe_rep1_annotated.h5ad")
    xe_rep2_annotated = sc.read(GENERATED_DATA_PATH / "xe_rep2_annotated.h5ad")

In [ ]:
xe_rep1_sdata.table.obs[["celltype_major", "celltype_minor"]] = xe_rep1_annotated.obs[
    ["celltype_major", "celltype_minor"]
]
xe_rep2_sdata.table.obs[["celltype_major", "celltype_minor"]] = xe_rep2_annotated.obs[
    ["celltype_major", "celltype_minor"]
]

#### Adding clonality information into Visium

In [ ]:
clones = clonal_adata.obs.set_index("barcode").loc[visium_sdata.table.obs.index]["clone"]
visium_sdata.table.obs["clone"] = clones

## Spatial alignment

### Alignment using 3 landmarks points

We take `xe_rep1` as the reference section and align the other two to it. The new coordinate systems is called `aligned`.

In [ ]:
xenium_subset = sd.SpatialData(
    images={
        "xe_rep1": xe_rep1_sdata.images["morphology_mip"],
        "xe_rep2": xe_rep2_sdata.images["morphology_mip"],
        "visium": visium_sdata.images["CytAssist_FFPE_Human_Breast_Cancer_full_image"],
    }
)

In [ ]:
# this creates the 'aligned` coordinate system, maps the moving (xenium rep 2) and reference elements (xenium rep 1) to that system
affine_rep2_to_rep1 = align_elements_using_landmarks(
    references_coords=landmarks_sdata.shapes["xe_rep1_lm"],
    moving_coords=landmarks_sdata.shapes["xe_rep2_lm"],
    reference_element=xenium_subset.images["xe_rep1"],
    moving_element=xenium_subset.images["xe_rep2"],
    reference_coordinate_system="global",
    moving_coordinate_system="global",
    new_coordinate_system="aligned",
)

In [ ]:
# same as above. Now the moving element is visium and the reference element is again xenium rep 1
affine_visium_to_rep1 = align_elements_using_landmarks(
    references_coords=landmarks_sdata.shapes["xe_rep1_lm"],
    moving_coords=landmarks_sdata.shapes["visium_lm"],
    reference_element=xenium_subset.images["xe_rep1"],
    moving_element=xenium_subset.images["visium"],
    reference_coordinate_system="global",
    moving_coordinate_system="global",
    new_coordinate_system="aligned",
)

For each element of xenium rep 1, xenium rep 2 and of visium, let's add a transformation to the coordinate system 'aligned'. This instruct the framework how to map each element to the 'aligned' coordinate system.

This part of the code will be simplified and become more ergonomic after the new coordinate systems refactoring.

In [ ]:
# we use an identity for xenium rep 1 since we use it as a reference
from spatialdata import SpatialData
from spatialdata.transformations import BaseTransformation


def postpone_transformation(
    sdata: SpatialData,
    transformation: BaseTransformation,
    source_coordinate_system: str,
    target_coordinate_system: str,
):
    for element_type, element_name, element in sdata._gen_elements():
        old_transformations = get_transformation(element, get_all=True)
        if source_coordinate_system in old_transformations:
            old_transformation = old_transformations[source_coordinate_system]
            sequence = Sequence([old_transformation, transformation])
            set_transformation(element, sequence, target_coordinate_system)


postpone_transformation(
    sdata=xe_rep1_sdata,
    transformation=Identity(),
    source_coordinate_system="global",
    target_coordinate_system="aligned",
)
postpone_transformation(
    sdata=xe_rep2_sdata,
    transformation=affine_rep2_to_rep1,
    source_coordinate_system="global",
    target_coordinate_system="aligned",
)
postpone_transformation(
    sdata=visium_sdata,
    transformation=affine_visium_to_rep1,
    source_coordinate_system="global",
    target_coordinate_system="aligned",
)

### Subsetting of the data
Used for debugging and dev purposes, remove later

### Subsetting the objects to the common area

We now want to subset each object to the common area between the two Xenium replicates and the Visium data. Currently there is no function available in `spatialdata` to do a spatial subset by a generic polygon (it will be implemented), so for performing this opertation we will manually transform the data to the same coordinate system, and then implement here a version of the spatial subset.

#### Transforming cells and single-molecule points

We will now transform the data to the `aligned` coordinate system. Note that above we just defined transformations to this coordinate system but we didn't modified the data itself (this is an expensive operation). Let's create new objects that don't contain the large images.

In [ ]:
xe_rep1_transformed_sdata = sd.SpatialData(
    shapes=xe_rep1_sdata.shapes, points=xe_rep1_sdata.points, table=xe_rep1_sdata.table
)

xe_rep2_transformed_sdata = sd.SpatialData(
    shapes=xe_rep2_sdata.shapes, points=xe_rep2_sdata.points, table=xe_rep2_sdata.table
)

visium_transformed_sdata = sd.SpatialData(
    shapes=visium_sdata.shapes, points=visium_sdata.points, table=visium_sdata.table
)

xe_rep1_transformed_sdata = xe_rep1_transformed_sdata.transform_to_coordinate_system("aligned")
xe_rep2_transformed_sdata = xe_rep2_transformed_sdata.transform_to_coordinate_system("aligned")
visium_transformed_sdata = visium_transformed_sdata.transform_to_coordinate_system("aligned")

#### Getting the polygon describing the common area

In [ ]:
from geopandas import GeoSeries


def get_extent(geoseries: GeoSeries):
    min_x, min_y = np.min(geoseries.bounds.iloc[:, :2], axis=0)
    max_x, max_y = np.max(geoseries.bounds.iloc[:, 2:], axis=0)
    print(f"min_x = {min_x}, min_y = {min_y}, max_x = {max_x}, max_y = {max_y}")

In [ ]:
# we manually draw the polygon that contains the common area between the samples
box = landmarks_sdata.shapes["box"]
# let's get the Polygon object out of the GeoDataFrame
box = box.geometry.iloc[0]
box

#### Filtering the data inside the common area

The functions below implement spatial queries operations on polygons. These functions will be implemented in SpatialData and the code will become simpler.

In [ ]:
from spatialdata import polygon_query

In [ ]:
%%time
# let's keep only the shapes inside the query polygon (roi)
xe_rep1_roi_sdata = polygon_query(
    sdata=xe_rep1_transformed_sdata, polygons=box, target_coordinate_system="aligned", filter_table=False
)

In [ ]:
%%time
xe_rep2_roi_sdata = polygon_query(
    sdata=xe_rep2_transformed_sdata, polygons=box, target_coordinate_system="aligned", filter_table=False
)

In [ ]:
%%time
visium_roi_sdata = polygon_query(
    sdata=visium_transformed_sdata,
    polygons=box,
    target_coordinate_system="aligned",
)

### Subsetting the objects to the common genes

In [ ]:
sel_genes = list(set(visium_roi_sdata.table.var_names) & (set(xe_rep1_roi_sdata.table.var_names)))

filtered_table = xe_rep1_roi_sdata.table[:, sel_genes].copy()
del xe_rep1_roi_sdata.table
xe_rep1_roi_sdata.table = filtered_table

filtered_table = xe_rep2_roi_sdata.table[:, sel_genes].copy()
del xe_rep2_roi_sdata.table
xe_rep2_roi_sdata.table = filtered_table

filtered_table = visium_roi_sdata.table[:, sel_genes].copy()
del visium_roi_sdata.table
visium_roi_sdata.table = filtered_table

## Aggregation of gene expression and cell-types from the Xenium cells into the Visium circles

We will now aggregate the gene expression and the cell-type information and the into the Visium circles.

To do so, we will consider the polygonal cell description. Currently the table is set to describe the cell circles (as we can see in the next cell). Therefore, let's create another `SpatialData` object that links the table to the polygonal description, and let's use this for aggregation. Let's do this for both replicates.

The option `fractions=True` is used to downweigh cases of partial cell overlaps, please refer to the documentation for a detailed explanation.

In [ ]:
from spatialdata.models import TableModel

table_metadata = xe_rep1_roi_sdata.table.uns[TableModel.ATTRS_KEY]
print(table_metadata)

# replicate 1
new_table = xe_rep1_roi_sdata.table.copy()
del new_table.uns[TableModel.ATTRS_KEY]
new_table.obs["region"] = "cell_boundaries"
new_table.obs["region"] = new_table.obs["region"].astype("category")
new_table = TableModel.parse(new_table, region="cell_boundaries", region_key="region", instance_key="cell_id")
xe_rep1_roi_sdata_polygons = SpatialData(
    shapes={"cell_boundaries": xe_rep1_roi_sdata["cell_boundaries"]}, table=new_table
)

# replicate 2
new_table = xe_rep2_roi_sdata.table.copy()
del new_table.uns[TableModel.ATTRS_KEY]
new_table.obs["region"] = "cell_boundaries"
new_table.obs["region"] = new_table.obs["region"].astype("category")
new_table = TableModel.parse(new_table, region="cell_boundaries", region_key="region", instance_key="cell_id")
xe_rep2_roi_sdata_polygons = SpatialData(
    shapes={"cell_boundaries": xe_rep2_roi_sdata["cell_boundaries"]}, table=new_table
)

### Aggregating gene expression.

In [ ]:
%%time
# rep 1
agg_expression = aggregate(
    values_sdata=xe_rep1_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key=xe_rep1_roi_sdata.table.var_names.tolist(),
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.layers["xe_rep1_cells"] = agg_expression.table[:, sel_genes].X.copy()

# rep 2
agg_expression = aggregate(
    values_sdata=xe_rep2_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key=xe_rep2_roi_sdata.table.var_names.tolist(),
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.layers["xe_rep2_cells"] = agg_expression.table[:, sel_genes].X.copy()

### Aggregating cell-types into cell-type fractions

#### Major cell-types

In [ ]:
%%time
# rep 1
agg_celltype_major = aggregate(
    values_sdata=xe_rep1_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key="celltype_major",
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.obsm["xe_rep1_celltype_major"] = agg_celltype_major.table.X.A

# rep 2
agg_celltype_major = aggregate(
    values_sdata=xe_rep2_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key="celltype_major",
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.obsm["xe_rep2_celltype_major"] = agg_celltype_major.table.X.A

#### Minor cell-types

In [ ]:
%%time
# rep 1
agg_celltype_minor = aggregate(
    values_sdata=xe_rep1_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key="celltype_minor",
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.obsm["xe_rep1_celltype_minor"] = agg_celltype_minor.table.X.A

# rep 2
agg_celltype_minor = aggregate(
    values_sdata=xe_rep2_roi_sdata_polygons,
    values="cell_boundaries",
    by_sdata=visium_roi_sdata,
    by="CytAssist_FFPE_Human_Breast_Cancer",
    value_key="celltype_minor",
    target_coordinate_system="aligned",
    fractions=True,
)
visium_roi_sdata.table.obsm["xe_rep2_celltype_minor"] = agg_celltype_minor.table.X.A

## Aggregation of gene expression from the Xenium single-molecule points into the Visium Circles

In [ ]:
%%time
xe_rep1_aggregated = aggregate(
    values=xe_rep1_roi_sdata["transcripts"],
    by=visium_roi_sdata["CytAssist_FFPE_Human_Breast_Cancer"],
    value_key="feature_name",
    agg_func="count",
    target_coordinate_system="aligned",
)

In [ ]:
counts = xe_rep1_aggregated.table[:, sel_genes].X.todense().A
visium_roi_sdata.table.layers["xe_rep1_tx"] = counts

In [ ]:
%%time
xe_rep2_aggregated = aggregate(
    values=xe_rep2_roi_sdata["transcripts"],
    by=visium_roi_sdata["CytAssist_FFPE_Human_Breast_Cancer"],
    value_key="feature_name",
    agg_func="count",
    target_coordinate_system="aligned",
)

In [ ]:
counts = xe_rep2_aggregated.table[:, sel_genes].X.todense().A
visium_roi_sdata.table.layers["xe_rep2_tx"] = counts

## Saving the objects to disk

In [ ]:
xe_rep1_roi_sdata

In [ ]:
xe_rep2_roi_sdata

In [ ]:
visium_roi_sdata

In [ ]:
# save transformations to disk (it was only in-memory so far)
from spatialdata import save_transformations

save_transformations(xe_rep1_sdata)
save_transformations(xe_rep2_sdata)
save_transformations(visium_sdata)

In [ ]:
# the table of the xenium data was modified only in-memory, let's sync the change to disk


def save_table(sdata: SpatialData) -> None:
    table = sdata.table
    del sdata.table
    # updates the disk storage
    sdata.table = table


save_table(xe_rep1_sdata)
save_table(xe_rep2_sdata)
save_table(visium_sdata)

In [ ]:
xe_rep1_roi_sdata["transcripts"]["feature_name"] = xe_rep1_roi_sdata["transcripts"]["feature_name"].cat.as_known()
xe_rep2_roi_sdata["transcripts"]["feature_name"] = xe_rep2_roi_sdata["transcripts"]["feature_name"].cat.as_known()

In [ ]:
%%time
import shutil

if XE_REP1_ROI_PATH.is_dir():
    shutil.rmtree(XE_REP1_ROI_PATH)
xe_rep1_roi_sdata.write(XE_REP1_ROI_PATH)

if XE_REP2_ROI_PATH.is_dir():
    shutil.rmtree(XE_REP2_ROI_PATH)
xe_rep2_roi_sdata.write(XE_REP2_ROI_PATH)

if VISIUM_ROI_PATH.is_dir():
    shutil.rmtree(VISIUM_ROI_PATH)
visium_roi_sdata.write(VISIUM_ROI_PATH)

### Internal code

During development, we conclude doing a check that the data that we saved is correct.

In [ ]:
xe_rep1_sdata2 = sd.read_zarr(XE_REP1_PATH)
xe_rep2_sdata2 = sd.read_zarr(XE_REP2_PATH)
visium_sdata2 = sd.read_zarr(VISIUM_PATH)

xe_rep1_roi_sdata2 = sd.read_zarr(XE_REP1_ROI_PATH)
xe_rep2_roi_sdata2 = sd.read_zarr(XE_REP2_ROI_PATH)
visium_roi_sdata2 = sd.read_zarr(VISIUM_ROI_PATH)

In [ ]:
assert str(xe_rep1_sdata) == str(xe_rep1_sdata2)
assert str(xe_rep2_sdata) == str(xe_rep2_sdata2)
assert str(visium_sdata) == str(visium_sdata2)

assert str(xe_rep1_roi_sdata) == str(xe_rep1_roi_sdata2)
assert str(xe_rep2_roi_sdata) == str(xe_rep2_roi_sdata2)
assert sorted(str(visium_roi_sdata)) == sorted(str(visium_roi_sdata2))

For interal pipelines used during development, we also copy the produced data into a new location.

In [ ]:
XE_REP1_ALIGNED_PATH = XE_REP1_PATH.with_name("data_aligned.zarr")
XE_REP2_ALIGNED_PATH = XE_REP2_PATH.with_name("data_aligned.zarr")
VISIUM_ALIGNED_PATH = VISIUM_PATH.with_name("data_aligned.zarr")

if XE_REP1_ALIGNED_PATH.is_dir():
    shutil.rmtree(XE_REP1_ALIGNED_PATH)
shutil.copytree(XE_REP1_PATH, XE_REP1_ALIGNED_PATH)

if XE_REP2_ALIGNED_PATH.is_dir():
    shutil.rmtree(XE_REP2_ALIGNED_PATH)
shutil.copytree(XE_REP2_PATH, XE_REP2_ALIGNED_PATH)

if VISIUM_ALIGNED_PATH.is_dir():
    shutil.rmtree(VISIUM_ALIGNED_PATH)
shutil.copytree(VISIUM_PATH, VISIUM_ALIGNED_PATH)
